In [2]:
import matplotlib.pyplot as plt
import json
import os
import cv2
import numpy as np
import random
from random import shuffle
import os
from skimage import transform

In [3]:
"""Prepare Data"""
images_path = "eccv_18_all_images_sm"
labels_path = "CaltechCameraTrapsECCV18.json"

with open(labels_path) as json_file:
    data = json.load(json_file)
    
path_lst = next(os.walk(images_path))[2]

print(len(path_lst))

data['categories']

57864


[{'id': 6, 'name': 'bobcat'},
 {'id': 1, 'name': 'opossum'},
 {'id': 30, 'name': 'empty'},
 {'id': 9, 'name': 'coyote'},
 {'id': 3, 'name': 'raccoon'},
 {'id': 11, 'name': 'bird'},
 {'id': 8, 'name': 'dog'},
 {'id': 16, 'name': 'cat'},
 {'id': 5, 'name': 'squirrel'},
 {'id': 10, 'name': 'rabbit'},
 {'id': 7, 'name': 'skunk'},
 {'id': 99, 'name': 'rodent'},
 {'id': 21, 'name': 'badger'},
 {'id': 34, 'name': 'deer'},
 {'id': 33, 'name': 'car'},
 {'id': 51, 'name': 'fox'}]

In [4]:
clist, cdict = [], {}
for category in data['categories']:
    cdict[category['id']] = len(clist)
    clist.append(category['name'])

num_categories = len(clist)

adict = {}
for annotation in data['annotations']:
    adict[annotation['image_id']] = cdict[annotation['category_id']]

print(clist)
cdict

['bobcat', 'opossum', 'empty', 'coyote', 'raccoon', 'bird', 'dog', 'cat', 'squirrel', 'rabbit', 'skunk', 'rodent', 'badger', 'deer', 'car', 'fox']


{1: 1,
 3: 4,
 5: 8,
 6: 0,
 7: 10,
 8: 6,
 9: 3,
 10: 9,
 11: 5,
 16: 7,
 21: 12,
 30: 2,
 33: 14,
 34: 13,
 51: 15,
 99: 11}

In [5]:
dim = (512, 384)
def select(idx):
    path = path_lst[idx]
    
    imarr = cv2.imread(images_path + '/' + path, cv2.IMREAD_COLOR)
    imarr = cv2.resize(imarr, dim, interpolation = cv2.INTER_AREA)
    #imarr = transform.resize(imarr, dim, mode='reflect')
    
    label = adict[path.split('.')[0]]
    
    return imarr / 255, label

In [6]:
idx = random.randint(0, len(path_lst))
imarr, label = select(idx)
print(path_lst[idx])
print(np.max(imarr))
print(clist[label])
print(imarr.shape)
print(imarr.dtype)
plt.imshow(imarr)

5986246b-23d2-11e8-a6a3-ec086b02610b.jpg
1.0
car
(384, 512, 3)
float64


In [96]:
size_dict, label_dict = {}, {}
i = 0
for idx in range(len(path_lst)):
    imarr, label = select(idx)
    
    if imarr.shape in size_dict:
        size_dict[imarr.shape] += 1
    else:
        size_dict[imarr.shape] = 1
        
    if label in label_dict:
        label_dict[label] += 1
    else:
        label_dict[label] = 1
    
    if i % 500 == 0:
        print(i / len(path_lst) * 100, '% done')
    i+=1

print(label_dict)
size_dict

0.0 % done
0.8640951195907645 % done
1.728190239181529 % done
2.5922853587722936 % done
3.456380478363058 % done
4.3204755979538225 % done
5.184570717544587 % done
6.048665837135352 % done
6.912760956726116 % done
7.77685607631688 % done
8.640951195907645 % done
9.50504631549841 % done
10.369141435089174 % done
11.233236554679939 % done
12.097331674270704 % done
12.961426793861468 % done
13.825521913452231 % done
14.689617033042998 % done
15.55371215263376 % done
16.41780727222453 % done
17.28190239181529 % done
18.145997511406055 % done
19.01009263099682 % done
19.874187750587584 % done
20.73828287017835 % done
21.602377989769113 % done
22.466473109359878 % done
23.330568228950643 % done
24.194663348541408 % done
25.05875846813217 % done
25.922853587722937 % done
26.7869487073137 % done
27.651043826904463 % done
28.515138946495227 % done
29.379234066085996 % done
30.24332918567676 % done
31.10742430526752 % done
31.971519424858286 % done
32.83561454444906 % done
33.699709664039815 % d

{(747, 1024, 3): 52912, (768, 1024, 3): 4952}

In [12]:
image_lst, label_lst = [], []
shuffle(path_lst)

num_examples = 10000
channels = 3

images = np.zeros(dtype='float16', shape=(num_examples, dim[1], dim[0], channels))
labels = np.zeros(dtype='byte', shape=(num_examples, num_categories))

for example in range(num_examples):
    imarr, label = select(example)
    images[example] = imarr
    labels[example][label] = 1
    
    if (example % 500 == 0):
        print('finished {0} of {1}'.format(example + 1, num_examples))

finished 1 of 10000
finished 501 of 10000
finished 1001 of 10000
finished 1501 of 10000
finished 2001 of 10000
finished 2501 of 10000
finished 3001 of 10000
finished 3501 of 10000
finished 4001 of 10000
finished 4501 of 10000
finished 5001 of 10000
finished 5501 of 10000
finished 6001 of 10000
finished 6501 of 10000
finished 7001 of 10000
finished 7501 of 10000
finished 8001 of 10000
finished 8501 of 10000
finished 9001 of 10000
finished 9501 of 10000


In [13]:
np.max(images)

1.0

In [16]:
np.save('images.npy', images)
np.save('labels.npy', labels)
np.savez('label_to_animal.npz', clist)
np.save('small_images.npy', images[:1000])
np.save('small_labels.npy', labels[:1000])

In [17]:
images.shape

(10000, 384, 512, 3)